# Impacto de la clasificación por edad en popularidad
•	Hipótesis:
 El contenido clasificado como "TV-MA" tiene un mayor número de votos en IMDb en comparación con contenido para audiencias más jóvenes.

•	Objetivo:
 Determinar si se debe enfocar la producción en contenido para audiencias maduras o diversificar hacia segmentos más familiares.

•	Análisis: Comparar el número promedio de votos (imdb_votes) y calificaciones (imdb_score) por clasificación de edad (age_certification).


In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pymongo

In [ ]:
# Datos en formato CSV
data_netflix = pd.read_csv("data_netflix.csv")
mejores_shows = pd.read_csv("Mejores_shows_netflix.csv")
mejores_peliculas = pd.read_csv("Mejores_películas_Netflix.csv")
actores = pd.read_csv("Actores.csv")

In [ ]:
# Conexión a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Acceso a la BD
db = client["netflix"]

# Acceso a la colección
collection = db['data_netflix']

In [ ]:
pipeline = [
    {
        "$match": {
            "age_certification": { "$ne": None },  # Excluir documentos sin clasificación por edad
            "imdb_votes": { "$gt": 0 },             # Considerar solo contenido con votos válidos
            "imdb_score": { "$gt": 0 }              # Considerar solo contenido con calificación válida
        }
    },
    {
        "$group": {
            "_id": "$age_certification",            # Agrupar por clasificación de edad
            "avg_votes": { "$avg": "$imdb_votes" }, # Campo promedio de votos
            "avg_score": { "$avg": "$imdb_score" }, # Campo promedio de calificación
            "count": { "$sum": 1 }                  # Contar cuántos elementos hay por clasificación
        }
    },
    {
        "$sort": { "avg_votes": -1 }               # Ordenar por votos promedio de mayor a menor
    },
    {
        "$out": "H1:impacto_de_la_clasificación_por_edad_en_popularidad"  # Guardar los resultados en una nueva colección
    }
]

# Ejecutar la agregación
result = collection.aggregate(pipeline)


# Principales hallazgos:

En los resultados: Se sugiere enfocar la producción en contenido para audiencias maduras


En la limpieza: Se excluyeron documentos sin clasificación por edad, sin votos  y sin calificaciones.
Se consideraron solo contenido con votos  y calificaciones válidos (aunque no hay registros <= 0)
